In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
class LagrangianNN(nn.Module):
    def __init__(self, input_dim, hidden_layers):
        super(LagrangianNN, self).__init__()
        layers = []
        last_dim = input_dim
        for layer_size in hidden_layers:
            layers.append(nn.Linear(last_dim, layer_size))
            layers.append(nn.ReLU())
            last_dim = layer_size
        layers.append(nn.Linear(last_dim, 1))
        self.net = nn.Sequential(*layers)

    def forward(self, q, q_dot):
        inputs = torch.cat([q, q_dot], dim=1) # concatenate q and q_dot for the input
        return self.net(inputs)

In [ ]:
def lagrangian_loss(model, q, q_dot, q_dot_dot, actuator_forces):
    L = model(q, q_dot)
    L_grad = torch.autograd.grad(L.sum(), q_dot, create_graph=True)[0]
    
    predicted_q_dot_dot = torch.autograd.grad(L_grad.sum(), q, -actuator_forces, create_graph=True)[0]
    
    return ((predicted_q_dot_dot - q_dot_dot) ** 2).mean()


In [ ]:
class HamiltonianNN(nn.Module):
    def __init__(self, input_dim, hidden_layers):
        super(HamiltonianNN, self).__init__()
        layers = []
        last_dim = input_dim
        for layer_size in hidden_layers:
            layers.append(nn.Linear(last_dim, layer_size))
            layers.append(nn.ReLU())
            last_dim = layer_size
        layers.append(nn.Linear(last_dim, 1))
        self.net = nn.Sequential(*layers)

    def forward(self, q, p):
        inputs = torch.cat([q, p], dim=1) # concatenate q and p for the input
        return self.net(inputs)

In [ ]:
def hamiltonian_loss(model, q, p, dp_dt, actuator_forces):
    H = model(q, p)
    H_grad_q = torch.autograd.grad(H.sum(), q, create_graph=True)[0]
    H_grad_p = torch.autograd.grad(H.sum(), p, create_graph=True)[0]

    predicted_dp_dt = -H_grad_q + actuator_forces
    predicted_dq_dt = H_grad_p

    return ((predicted_dp_dt - dp_dt) ** 2).mean()

In [ ]:
def train(model, loss_function, optimizer, epochs, train_loader):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for data in train_loader:
            q, q_dot, q_dot_dot, actuator_forces = data # unpack data.  to do: modify with the data

            optimizer.zero_grad()
            loss = loss_function(model, q, q_dot, q_dot_dot, actuator_forces)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        print(f'Epoch {epoch}, Loss: {total_loss / len(train_loader)}')


In [ ]:
input_dim = 4 # to do: adjust to the robotics problem (q and q_dot dimensionality)
hidden_layers = [64, 64]

lnn = LagrangianNN(input_dim, hidden_layers)
hnn = HamiltonianNN(input_dim, hidden_layers)

optimizer_lnn = optim.Adam(lnn.parameters(), lr=0.001)
optimizer_hnn = optim.Adam(hnn.parameters(), lr=0.001)

train_loader = None # assuming usage of a DataLoader for the dataset, to do: replace with DataLoader

train(lnn, lagrangian_loss, optimizer_lnn, 100, train_loader)

train(hnn, hamiltonian_loss, optimizer_hnn, 100, train_loader)
